In [1]:
import pandas as pd
import csv
from datetime import datetime, timedelta
import numpy as np

In [2]:
def create_template_dataframe(start_year=2020, end_year=2024):
    counties= ['New York', 'Cook', 'Los Angeles'] #Cities to be focused on

    # Creating a date range of January of 2020 to 2024
    all_dates= []
    for year in range(start_year, end_year + 1):
        january_start= pd.Timestamp(f'{year}-01-01 00:00:00')
        january_end= pd.Timestamp(f'{year}-01-31 23:00:00')
        january_dates= pd.date_range(january_start, january_end, freq='H')
        all_dates.extend(january_dates)

    # Creating an empty DataFrame with all column heads
    rows= []
    for date in all_dates:
        for county in counties:
            rows.append({
                'County Name': county,
                'Date': date.date(),
                'Year': date.year,
                'Month': date.month,
                'Day': date.day,
                'Hour': date.hour,
                'PM2.5': np.nan,
                'PM10': np.nan,
                'NO2': np.nan
            })

    return pd.DataFrame(rows)

In [3]:
def remove_duplicates(data, parameter):
    #Removing duplicates of data based on the last date of change
    if 'Date of Last Change' in data.columns:
        data['Date of Last Change']= pd.to_datetime(data['Date of Last Change'])
        deduped_data= data.sort_values('Date of Last Change', ascending=False).drop_duplicates(
            subset=['County Name', 'Date Local', 'Time Local'],
            keep='first'
        )
    else:
        deduped_data = data.drop_duplicates(
            subset=['County Name', 'Date Local', 'Time Local'],
            keep='first'
        )

    return deduped_data

In [4]:
def process_parameter_data(raw_data, parameter):

    deduped_data= remove_duplicates(raw_data, parameter)
    deduped_data['DateTime']= pd.to_datetime(
        deduped_data['Date Local'] + ' ' + deduped_data['Time Local'])

    processed_data= deduped_data[['County Name', 'DateTime', 'Sample Measurement']]

    # Spliting the date and hour for easier processing
    processed_data['Date']= processed_data['DateTime'].dt.date
    processed_data['Hour']= processed_data['DateTime'].dt.hour

    return processed_data

In [5]:
def merge_parameter_data(template_df, processed_data, parameter):
    # Merging tables from the raw table to our dataset
    merged_df= template_df.merge(
        processed_data[['County Name', 'Date', 'Hour', 'Sample Measurement']],
        on=['County Name', 'Date', 'Hour'],
        how='left'
    )

    merged_df[parameter]= merged_df['Sample Measurement'].combine_first(merged_df[parameter])
    merged_df = merged_df.drop('Sample Measurement', axis=1)

    return merged_df

In [6]:
def process_all_data(data_files):
    years= list(data_files.keys())
    template_df= create_template_dataframe(min(years), max(years))

    final_df= template_df.copy()

    for year in data_files:
        print(f"\nProcessing year {year}")
        for parameter, file_path in data_files[year].items():
            print(f"Processing {parameter} data from {file_path}")

            # Reading and processing the data to keep in the final dataset
            raw_data= pd.read_csv(file_path, on_bad_lines='skip', engine='python')
            processed_data= process_parameter_data(raw_data, parameter)
            final_df= merge_parameter_data(final_df, processed_data, parameter)

            print(f"\nSuccessfully processed {parameter} data for {year}")

    final_df= final_df.sort_values(['County Name', 'Date', 'Hour'])

    return final_df

In [7]:
data_files = {
    2020: {
        'PM2.5': 'data/hourly_88502_2020.csv',
        'PM10': 'data/hourly_81102_2020.csv',
        'NO2': 'data/hourly_42602_2020.csv'
    },
    2021: {
        'PM2.5': 'data/hourly_88502_2021.csv',
        'PM10': 'data/hourly_81102_2021.csv',
        'NO2': 'data/hourly_42602_2021.csv'
    },
    2022: {
        'PM2.5': 'data/hourly_88502_2022.csv',
        'PM10': 'data/hourly_81102_2022.csv',
        'NO2': 'data/hourly_42602_2022.csv'
    },
    2023: {
        'PM2.5': 'data/hourly_88502_2023.csv',
        'PM10': 'data/hourly_81102_2023.csv',
        'NO2': 'data/hourly_42602_2023.csv'
    },
    2024: {
        'PM2.5': 'data/hourly_88502_2024.csv',
        'PM10': 'data/hourly_81102_2024.csv',
        'NO2': 'data/hourly_42602_2024.csv'
    },
}

final_df= process_all_data(data_files)

<ipython-input-2-bd9f6381e4ad>:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  january_dates= pd.date_range(january_start, january_end, freq='H')



Processing year 2020
Processing PM2.5 data from hourly_88502_2020.csv


<ipython-input-4-9b85bddf4ab3>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_data['Date']= processed_data['DateTime'].dt.date
<ipython-input-4-9b85bddf4ab3>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_data['Hour']= processed_data['DateTime'].dt.hour



Successfully processed PM2.5 data for 2020
Processing PM10 data from hourly_81102_2020.csv


<ipython-input-4-9b85bddf4ab3>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_data['Date']= processed_data['DateTime'].dt.date
<ipython-input-4-9b85bddf4ab3>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_data['Hour']= processed_data['DateTime'].dt.hour


...


Successfully processed PM2.5 data for 2024
Processing PM10 data from hourly_81102_2024.csv


<ipython-input-4-9b85bddf4ab3>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_data['Date']= processed_data['DateTime'].dt.date
<ipython-input-4-9b85bddf4ab3>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_data['Hour']= processed_data['DateTime'].dt.hour



Successfully processed PM10 data for 2024
Processing NO2 data from hourly_42602_2024.csv


<ipython-input-4-9b85bddf4ab3>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_data['Date']= processed_data['DateTime'].dt.date
<ipython-input-4-9b85bddf4ab3>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processed_data['Hour']= processed_data['DateTime'].dt.hour



Successfully processed NO2 data for 2024


In [8]:
print("\nFinal DataFrame:")
final_df


Final DataFrame:


,County Name,Date,Year,Month,Day,Hour,PM2.5,PM10,NO2
1,Cook,2020-01-01,2020,1,1,0,3.8,20.0,11.7
4,Cook,2020-01-01,2020,1,1,1,7.7,23.0,14.6
7,Cook,2020-01-01,2020,1,1,2,4.8,22.0,12.7
10,Cook,2020-01-01,2020,1,1,3,5.4,16.0,14.0
13,Cook,2020-01-01,2020,1,1,4,6.7,15.0,12.9
...,...,...,...,...,...,...,...,...,...
11145,New York,2024-01-31,2024,1,31,19,12.6,NaN,NaN
11148,New York,2024-01-31,2024,1,31,20,11.2,NaN,NaN
11151,New York,2024-01-31,2024,1,31,21,11.5,NaN,NaN
11154,New York,2024-01-31,2024,1,31,22,11.8,NaN,NaN


In [9]:
final_df.to_csv('combined_air_quality_data.csv', index=False)